# DCGAN



In [1]:
import os
import tensorflow
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

### Load the data

In [2]:
from tensorflow.keras.datasets import mnist
(xtr,_),_ = mnist.load_data()
xtr = xtr.reshape((-1,28,28,1)).astype('float32')/255

### Create a dataViewer

In [3]:
%matplotlib inline
from GAN.utils import dataViewer

In [4]:
def viewMnist(mtr):
    '''Return from a numpy array an Image'''
    mtr = mtr.reshape((28,28))
    mtr *= 255
    mtr = np.clip(mtr,0,255).astype('uint8')
    imshow(mtr,cmap='gray')
def saveMnist(mtr,path):
    '''Save as file the numpy array'''
    mtr = mtr.reshape((28,28))
    mtr *= 255
    mtr = np.clip(mtr,0,255).astype('uint8')
    Image.fromarray(mtr).resize((280,280)).save(path)

In [5]:
vwr = dataViewer(functionView=viewMnist,functionSave=saveMnist,path="D:/Project/DeepLearning/GAN/DCGAN/output/")

### Create a DCGAN

In [6]:
from GAN.GAN.SimpleGAN import DCGAN

In [7]:
from tensorflow.keras.layers import Input, Dense,Reshape, Conv2D
from tensorflow.keras.layers import BatchNormalization,Activation
from tensorflow.keras.layers import Conv2DTranspose,Flatten, LeakyReLU
#Discriminator

kernel_size = 5
layer_filters = [32, 64, 128, 256]
inpDis = Input(shape=(28,28,1))
x = inpDis
for filters in layer_filters:
    # first 3 convolution layers use strides = 2
    # last one uses strides = 1
    if filters == layer_filters[-1]:
        strides = 1
    else:
        strides = 2
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(x)
x = Flatten()(x)
x = Dense(1)(x)
x = Activation('sigmoid')(x)
discriminator = Model(inpDis, x, name='discriminator')
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)       

In [8]:
#Generator

kernel_size = 5
layer_filters = [128, 64, 32, 1]

inputs = Input(shape=100)

x = Dense(7 * 7 * layer_filters[0])(inputs)
x = Reshape((7,7, layer_filters[0]))(x)

for filters in layer_filters:
    # first two convolution layers use strides = 2
    # the last two use strides = 1
    if filters > layer_filters[-2]:
        strides = 2
    else:
        strides = 1
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(x)
x = Activation('sigmoid')(x)
generator = Model(inputs, x, name='generator')
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 128)         512       
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       409728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       51

In [9]:
gan = DCGAN(generator=generator,discriminator=discriminator,DiscrOptimizer=RMSprop(lr=2e-4, decay=6e-8),GanOptimizer=RMSprop(lr=1e-4, decay=3e-8))

### Train the DCGAN

In [10]:
gan.generateBatchEval(16)

In [11]:
gan.GenerateOutputs(xtest=gan.evaluationInpt,batchSize=16,returnArray=False,dataViewer=vwr,save=True,View=False,epoch=0)

In [12]:
gan.train(x_train=xtr,batch_size=1024,epoch=30,evalStep=(1,10),dataViewer=vwr,pathSave='D:/Project/DeepLearning/GAN/DCGAN/save')

Epoch 1
117/117 [==============================] - 122s 1s/step
Discrimiator: (loss,acc)=(0.0006986771477386355,1.0)
 	 Adversial: (loss,acc)=(2.6589273716126627e-07,1.0)
Epoch 2
117/117 [==============================] - 102s 873ms/step
Discrimiator: (loss,acc)=(0.00021067119087092578,1.0)
 	 Adversial: (loss,acc)=(7.358487129211426,0.0)
Epoch 3
117/117 [==============================] - 104s 888ms/step
Discrimiator: (loss,acc)=(0.11280561238527298,0.99609375)
 	 Adversial: (loss,acc)=(9.659490585327148,0.0)
Epoch 4
117/117 [==============================] - 107s 916ms/step
Discrimiator: (loss,acc)=(0.0841422900557518,0.99609375)
 	 Adversial: (loss,acc)=(0.718354344367981,0.478515625)
Epoch 5
117/117 [==============================] - 105s 899ms/step
Discrimiator: (loss,acc)=(0.329184353351593,0.888671875)
 	 Adversial: (loss,acc)=(0.3071974813938141,0.97265625)
Epoch 6
117/117 [==============================] - 104s 890ms/step
Discrimiator: (loss,acc)=(0.6408595442771912,0.62109375)